<a href="https://colab.research.google.com/github/sweetpand/Algorithms/blob/master/MovePytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
from torch.autograd import Variable

import pandas as pd
from sklearn.model_selection import train_test_split

from PIL import Image

import sys, os, shutil, random

In [0]:
train_info = pd.read_csv("processed_train_info.csv")

processed_train_info = train_info.drop(['title', 'style', 'genre', 'date', 'Unnamed: 6', 'Unnamed: 0'], axis=1)

filenames = processed_train_info['filename'].tolist()
artists = processed_train_info['artist'].tolist()

artist_dict = {}
for i in range(len(artists)):
    if artists[i] not in artist_dict:
        artist_dict[artists[i]] = [filenames[i]]
    else:
        artist_dict[artists[i]].append(filenames[i])

In [0]:
def pick_positive_pair(artists, artist_dict):
	artist = np.random.choice(artists)
	return np.random.choice(artist_dict[artist], 2), artist

In [0]:
def pick_negative_pair(artists, artist_dict):
    artist_pair = np.random.choice(artists, 2)
    return [np.random.choice(artist_dict[artist_pair[0]]), np.random.choice(artist_dict[artist_pair[1]])]

In [0]:
pick_positive_pair(artists, artist_dict)

In [0]:
pick_negative_pair(artists, artist_dict)

In [0]:
class shallowCNN(nn.Module):
    def __init__(self):
        #self.config = config
        super(shallowCNN, self).__init__()
        # [in, out, kernel_size, stride, padding]
        self.bn0 = nn.BatchNorm2d(3)
        self.max_pool0 = nn.MaxPool2d(2, 2)

        self.conv1 = nn.Conv2d(3, 16, 3, 1, 1)
        self.bn1 = nn.BatchNorm2d(16)
        self.max_pool1 = nn.MaxPool2d(2, 2)

        self.conv2 = nn.Conv2d(16, 32, 3, 1, 1)
        self.bn2 = nn.BatchNorm2d(32)
        self.max_pool2 = nn.MaxPool2d(2, 2)

        self.conv3 = nn.Conv2d(32, 64, 3, 1, 1)
        self.bn3 = nn.BatchNorm2d(64)
        self.max_pool3 = nn.MaxPool2d(2, 2)
        
        self.conv4 = nn.Conv2d(64, 128, 3, 1, 1)
        self.bn4 = nn.BatchNorm2d(128)
        self.max_pool4 = nn.MaxPool2d(2, 2)

        self.conv5 = nn.Conv2d(128, 256, 3, 1, 1)
        self.bn5 = nn.BatchNorm2d(256)
        self.max_pool5 = nn.MaxPool2d(2, 2)
        self.linear1 = nn.Linear(256 * 4 * 4 * 2, 2048)

        self.linear2 = nn.Linear(2048, 346)
    def forward(self, x, xx):
        x = self.max_pool0(self.bn0(x))
        x = self.max_pool1(F.leaky_relu(self.bn1(self.conv1(x))))
        x = self.max_pool2(F.leaky_relu(self.bn2(self.conv2(x))))
        x = self.max_pool3(F.leaky_relu(self.bn3(self.conv3(x))))
        x = self.max_pool4(F.leaky_relu(self.bn4(self.conv4(x))))
        x = self.max_pool5(F.leaky_relu(self.bn5(self.conv5(x))))
        #print(x.size(), x.size(1) * x.size(2) * x.size(3))
        #exit()
        x = x.view(-1, x.size(1) * x.size(2) * x.size(3))
       
        xx = self.max_pool0(self.bn0(xx))
        xx = self.max_pool1(F.leaky_relu(self.bn1(self.conv1(xx))))
        xx = self.max_pool2(F.leaky_relu(self.bn2(self.conv2(xx))))
        xx = self.max_pool3(F.leaky_relu(self.bn3(self.conv3(xx))))
        xx = self.max_pool4(F.leaky_relu(self.bn4(self.conv4(xx))))
        xx = self.max_pool5(F.leaky_relu(self.bn5(self.conv5(xx))))
        xx = xx.view(-1, xx.size(1) * xx.size(2) * xx.size(3)) #[M, 4096]

        xxx = torch.cat([x, xx], dim=1) #[M, 4096 * 2]
        xxx = F.leaky_relu(self.linear1(xxx))
        xxx = F.dropout(xxx, p=0.5)
        xxx = F.sigmoid(xxx)
        return x

In [0]:
model = shallowCNN()

In [0]:
submission_info = pd.read_csv("submission_info.csv", index_col=['index'])

In [0]:
for i in range(len(submission_info["img1"])):
    im1 = np.array(Image.open("resized_test/" + submission_info["img1"][i]))
    im2 = np.array(Image.open("resized_test/" + submission_info["img2"][i]))

In [0]:
d = {0:1, 1:2}

In [0]:
s = pd.Series(d)

In [0]:
s.to_csv("test.csv", index_label="index", header=["h1"])